In [12]:
import numpy as np
import  math

layer1=np.zeros(2) # H
layer2=np.zeros(1)
# layer1_weight = [0.5, 0.5, 0.5 ,0.5]
# layer2_weight = [0.5, 0.5]
layer1_weight=[0.67974193, 0.26334452, 0.82376482, 0.4137218 ]
layer2_weight=[-0.51642946, -0.85370081]
# layer1_weight = np.random.rand(4)
# layer2_weight = np.random.rand(2)
bias=[0.3, 0.4]
learnRate=0.5

In [13]:
def sigmoid(H):
    return 1/(1+math.pow(math.e,-H))

In [14]:
# generate random data-set
np.random.seed(0)
noise = np.random.rand(100, 1)
rng = np.random.RandomState(1)
X = rng.rand(100, 2)
Y = 0.5 + np.dot(X, [1.5, -1.])

In [20]:

eTotal=0
for n in range(100):
    x=X[n]
    y=[Y[n]]
    # Forward Pass
    ## First layer
    for i in range(2):
        for j in range(2):
            layer1[i] += (x[j]*layer1_weight[(i*2)+j])

    ### calc first layer active function
    for i in range(2):
        layer1[i]=sigmoid(layer1[i]+bias[0])

    ## Second layer
    for i in range(1):
        for j in range(2):
            layer2[i] += (layer1[j]*layer2_weight[(i*2)+j])
    ### calc second layer active function
    for i in range(1):
        layer2[i]=sigmoid(layer2[i]+bias[1])

    ## calculating error
    eTotal+=(0.5*math.pow(layer2[0]-y[0],2))

    print(layer2)
    print(y)


[0.40520851]
[0.40520851361170285]
[0.42912712]
[0.19783898959417756]
[0.43693147]
[0.6277952414568717]
[0.43211894]
[0.4338295900234586]
[0.41791347]
[0.556334477342648]
[0.40954054]
[0.44357227120818277]
[0.4112484]
[-0.07143906179366932]
[0.42281831]
[-0.12938612038151298]
[0.41416596]
[0.5672673751049387]
[0.42801339]
[0.5124789188079719]
[0.39834532]
[0.7328552772939074]
[0.40587543]
[0.2778136515695502]
[0.39026898]
[0.91997706494021]
[0.42450722]
[0.5885115338217846]
[0.41923279]
[-0.12339687408255973]
[0.42736107]
[0.22641262574452298]
[0.3991297]
[1.4036690102527358]
[0.39952024]
[1.222300039919647]
[0.39366036]
[0.6951257196250026]
[0.41500788]
[-0.22271189892867982]
[0.39230728]
[1.2351259789799027]
[0.40362665]
[0.13138665964511925]
[0.42153712]
[0.20694548369055787]
[0.40195831]
[1.569279106265964]
[0.41819813]
[0.8016344357612455]
[0.42609902]
[-0.14978509613444535]
[0.42864219]
[0.5518955146278623]
[0.42188278]
[1.183997193803427]
[0.41442966]
[1.2144478333322093]
[0.402